#### Useful link:
https://spacy.io/usage/examples#section-training

In [204]:
import spacy
import pandas as pd
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import os

In [210]:
def read_datasets():
    return (pd.read_csv('../datasets/TrainNER.csv', delimiter=';', encoding='cp1252'), 
            pd.read_csv('../datasets/Test1NER.csv', names=['Sentence #', 'Word', 'POS', 'Tag'], delimiter=';', encoding='cp1252'), 
            pd.read_csv('../datasets/Test2NER.csv', names=['Sentence #', 'Word', 'POS', 'Tag'], delimiter=';', encoding='cp1252'))

def create_spacy_format_data(df):
    data = []
    for index, row in df.iterrows():
        if index < 1000:
            data.append((str(row['Word']), {'entities': [(0, len(str(row['Word'])), str(row['Tag']))]}))
        else: break
    return data
        
def init_data():
    train, test, test2 = read_datasets()
    print(train.columns)
    print(train.Tag.unique())
    return (create_spacy_format_data(train),
            create_spacy_format_data(test))
#     TEST2_DATA = create_spacy_format_data(test2)


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, output_dir=None, n_iter=100):
    TRAIN_DATA, TEST_DATA = init_data()
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print('Losses', losses)

    # test the trained model
    print("================TEST==========")
    for text, _ in TEST_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
#     if output_dir is not None:
#         output_dir = Path(output_dir)
#         if not output_dir.exists():
#             output_dir.mkdir()
#         nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)

        # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)

#     for text, _ in TEST_DATA:
#         doc = nlp(text)
#         print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#         print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == '__main__':
    main(output_dir='')

Index(['Sentence #', 'Word', 'POS', 'Tag'], dtype='object')
['B-per' 'I-per' 'O' 'B-org' 'B-gpe' 'I-org' 'B-geo' 'I-geo' 'B-time'
 'I-time' 'I-gpe' 'B-obj' 'I-obj' 'B-event' 'I-event' nan]
Created blank 'en' model
Losses {'ner': 22.40148496576722}
Losses {'ner': 15.604965789208665}
Losses {'ner': 13.61118188323601}
Losses {'ner': 13.218861905075453}
Losses {'ner': 10.409557446754034}
Losses {'ner': 8.585881673699134}
Losses {'ner': 7.891694834614505}
Losses {'ner': 6.264976266974879}
Losses {'ner': 6.469181890166241}
Losses {'ner': 6.1273440890608155}
Losses {'ner': 4.867422555473809}
Losses {'ner': 5.069322359610923}
Losses {'ner': 5.287714136238185}
Losses {'ner': 4.377111655542323}
Losses {'ner': 5.114151531462849}
Losses {'ner': 4.035326981714203}
Losses {'ner': 5.175788698777589}
Losses {'ner': 4.28078658674457}
Losses {'ner': 3.7401322748816495}
Losses {'ner': 4.215860729632144}
Losses {'ner': 4.488083859409313}
Losses {'ner': 3.601584985641444}
Losses {'ner': 3.656680904596946}


Entities [('HIV', 'O')]
Tokens [('HIV', 'O', 3)]
Entities [('/', 'O')]
Tokens [('/', 'O', 3)]
Entities [('AIDS', 'I-geo')]
Tokens [('AIDS', 'I-geo', 3)]
Entities [('burden', 'O')]
Tokens [('burden', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('and', 'O')]
Tokens [('and', 'O', 3)]
Entities [('market', 'O')]
Tokens [('market', 'O', 3)]
Entities [('distorting', 'O')]
Tokens [('distorting', 'O', 3)]
Entities [('agricultural', 'O')]
Tokens [('agricultural', 'O', 3)]
Entities [('policies', 'O')]
Tokens [('policies', 'O', 3)]
Entities [('have', 'O')]
Tokens [('have', 'O', 3)]
Entities [('meant', 'O')]
Tokens [('meant', 'O', 3)]
Entities [('that', 'O')]
Tokens [('that', 'O', 3)]
Entities [('Zambia', 'I-per')]
Tokens [('Zambia', 'I-per', 3)]
Entities [("'s", 'O')]
Tokens [("'s", 'O', 3)]
Entities [('economic', 'O')]
Tokens [('economic', 'O', 3)]
Entities [('growth', 'O')]
Tokens [('growth', 'O', 3)]
Entities [('has', 'O')]
Tokens [('has', 'O', 3)]
Entities [('not', 'O')]
To

Entities [('Czechs', 'I-org')]
Tokens [('Czechs', 'I-org', 3)]
Entities [('agreed', 'O')]
Tokens [('agreed', 'O', 3)]
Entities [('to', 'O')]
Tokens [('to', 'O', 3)]
Entities [('separate', 'O')]
Tokens [('separate', 'O', 3)]
Entities [('peacefully', 'O')]
Tokens [('peacefully', 'O', 3)]
Entities [('on', 'O')]
Tokens [('on', 'O', 3)]
Entities [('1', 'O')]
Tokens [('1', 'O', 3)]
Entities [('January', 'I-per')]
Tokens [('January', 'I-per', 3)]
Entities [('1993', 'O')]
Tokens [('1993', 'O', 3)]
Entities [('.', 'O')]
Tokens [('.', 'O', 3)]
Entities [('Slovakia', 'B-geo')]
Tokens [('Slovakia', 'B-geo', 3)]
Entities [('joined', 'O')]
Tokens [('joined', 'O', 3)]
Entities [('both', 'O')]
Tokens [('both', 'O', 3)]
Entities [('NATO', 'B-org')]
Tokens [('NATO', 'B-org', 3)]
Entities [('and', 'O')]
Tokens [('and', 'O', 3)]
Entities [('the', 'O')]
Tokens [('the', 'O', 3)]
Entities [('EU', 'I-org')]
Tokens [('EU', 'I-org', 3)]
Entities [('in', 'O')]
Tokens [('in', 'O', 3)]
Entities [('the', 'O')]
Toke

Entities [('country', 'O')]
Tokens [('country', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('but', 'O')]
Tokens [('but', 'O', 3)]
Entities [('also', 'O')]
Tokens [('also', 'O', 3)]
Entities [('has', 'O')]
Tokens [('has', 'O', 3)]
Entities [('strong', 'O')]
Tokens [('strong', 'O', 3)]
Entities [('ties', 'O')]
Tokens [('ties', 'O', 3)]
Entities [('with', 'O')]
Tokens [('with', 'O', 3)]
Entities [('the', 'O')]
Tokens [('the', 'O', 3)]
Entities [('US', 'B-gpe')]
Tokens [('US', 'B-gpe', 3)]
Entities [('.', 'O')]
Tokens [('.', 'O', 3)]
Entities [('Djibouti', 'B-per')]
Tokens [('Djibouti', 'B-per', 3)]
Entities [('hosts', 'O')]
Tokens [('hosts', 'O', 3)]
Entities [('the', 'O')]
Tokens [('the', 'O', 3)]
Entities [('only', 'O')]
Tokens [('only', 'O', 3)]
Entities [('US', 'B-gpe')]
Tokens [('US', 'B-gpe', 3)]
Entities [('military', 'O')]
Tokens [('military', 'O', 3)]
Entities [('base', 'O')]
Tokens [('base', 'O', 3)]
Entities [('in', 'O')]
Tokens [('in', 'O', 3)]
Entities [(

Entities [('including', 'O')]
Tokens [('including', 'O', 3)]
Entities [('onion', 'O')]
Tokens [('onion', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('chili', 'O')]
Tokens [('chili', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('beer', 'O')]
Tokens [('beer', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('eggplant', 'O')]
Tokens [('eggplant', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('smoked', 'O')]
Tokens [('smoked', 'O', 3)]
Entities [('trout', 'O')]
Tokens [('trout', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('spaghetti', 'O')]
Tokens [('spaghetti', 'O', 3)]
Entities [('parmesan', 'O')]
Tokens [('parmesan', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('chicken', 'O')]
Tokens [('chicken', 'O', 3)]
Entities [('with', 'O')]
Tokens [('with', 'O', 3)]
Entities [('rice', 'O')]
Tokens [('rice', 'O', 3)]
Entities [(',', 'O')]
Tokens [(',', 'O', 3)]
Entities [('and', 'O'

Entities [('custodian', 'O')]
Tokens [('custodian', 'O', 3)]
Entities [('who', 'O')]
Tokens [('who', 'O', 3)]
Entities [('had', 'O')]
Tokens [('had', 'O', 3)]
Entities [('to', 'O')]
Tokens [('to', 'O', 3)]
Entities [('clean', 'O')]
Tokens [('clean', 'O', 3)]
Entities [('the', 'O')]
Tokens [('the', 'O', 3)]
Entities [('mirrors', 'O')]
Tokens [('mirrors', 'O', 3)]
Entities [('every', 'O')]
Tokens [('every', 'O', 3)]
Entities [('night', 'O')]
Tokens [('night', 'O', 3)]
Entities [('.', 'O')]
Tokens [('.', 'O', 3)]
Entities [('To', 'O')]
Tokens [('To', 'O', 3)]
Entities [('demonstrate', 'O')]
Tokens [('demonstrate', 'O', 3)]
Entities [('how', 'O')]
Tokens [('how', 'O', 3)]
Entities [('difficult', 'O')]
Tokens [('difficult', 'O', 3)]
Entities [('it', 'O')]
Tokens [('it', 'O', 3)]
Entities [('had', 'O')]
Tokens [('had', 'O', 3)]
Entities [('been', 'O')]
Tokens [('been', 'O', 3)]
Entities [('to', 'O')]
Tokens [('to', 'O', 3)]
Entities [('clean', 'O')]
Tokens [('clean', 'O', 3)]
Entities [('the

Entities [('another', 'O')]
Tokens [('another', 'O', 3)]
Entities [('38', 'O')]
Tokens [('38', 'O', 3)]
Entities [('opposition', 'O')]
Tokens [('opposition', 'O', 3)]
Entities [('members', 'O')]
Tokens [('members', 'O', 3)]
Entities [('last', 'O')]
Tokens [('last', 'O', 3)]
Entities [('month', 'O')]
Tokens [('month', 'O', 3)]
Entities [('.', 'O')]
Tokens [('.', 'O', 3)]
Entities [('That', 'I-org')]
Tokens [('That', 'I-org', 3)]
Entities [('group', 'O')]
Tokens [('group', 'O', 3)]
Entities [('had', 'O')]
Tokens [('had', 'O', 3)]
Entities [('received', 'O')]
Tokens [('received', 'O', 3)]
Entities [('life', 'O')]
Tokens [('life', 'O', 3)]
Entities [('sentences', 'O')]
Tokens [('sentences', 'O', 3)]
Entities [('in', 'O')]
Tokens [('in', 'O', 3)]
Entities [('court', 'O')]
Tokens [('court', 'O', 3)]
Entities [('the', 'O')]
Tokens [('the', 'O', 3)]
Entities [('week', 'O')]
Tokens [('week', 'O', 3)]
Entities [('before', 'O')]
Tokens [('before', 'O', 3)]
Entities [('their', 'O')]
Tokens [('thei